In [1]:
import pandas as pd
import numpy as np
import glob

## Form Preprocessing

Before running the pipeline, I will first update the secret code dictionary as the quarter is going. In WI24, the professor used google form to collect participation forms from students. After downloading the spreadsheet that contains the submission entries, I will do the following steps: 
1. Rename the filename to the date of lecture. For example, "B&B: Week 2 (Wednesday 1/17/24) (Responses).xlsx" --> "1-17-24" so that pipeline can use the filename to identiy which day the record belongs to
2. Move the file to the folder for that week (i.e. a folder named "Week2" for "1/17/24.xlsx")

You can find the participation record for WI24 hereas an example: https://docs.google.com/spreadsheets/d/1-ZvP1N-38dD1XISa0--tas1ch_N70zTAr5YRZWtNTow/edit?usp=sharing. Noted that the submission history for each lecture day was mannually added as reference in case students think there is error in their record. 

In [2]:
secret_code = {
    '1-17-24': 'shadedregion',
    '1-19-24': 'coffee',
    '1-22-24': 'flood',
    '1-24-24': 'shiny',
    '1-26-24': 'elbow',
    '1-29-24': 'dutton',
    '1-31-24': 'anayake',
    '2-2-24': 'remote',
    '2-5-24': 'bayesian',
    '2-7-24': 'waldo',
    '2-9-24': 'brain',
    '2-14-24': 'voodoo',
    '2-16-24': 'bigrockcandymountain',
    '2-21-24': 'week7sucks!',
    '2-23-24': 'bloom',
    '2-28-24': 'sword',
    '3-1-24': 'sundance',
    '3-6-24': 'march15',
    '3-8-24': 'estrogen',
    '3-11-24': 'cookies',
    '3-13-24': 'binomial',
    '3-15-24': 'almostspringbreak!'
}

#Project day participation forms have different format than usual
project_day = ['./Week6/2-12-24.xlsx', './Week8/2-26-24.xlsx', './Week9/3-4-24.xlsx']
remote = 'remote'

In [7]:
#the roaster file here is essentially a raw gradebook I downloaded from canvas at the beginning of the quarter
roaster = pd.read_csv('2024-01-22T0111_Grades-COGS111_WI24_A00.csv')
roaster['SIS User ID'] = roaster['SIS User ID'].astype('string')

## Week 1 Participation

Since everyone is still joining the class, every body who submitted the form got participation credits in week 1

In [8]:
file_names = glob.glob('./Week1/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    files.append(temp_file)

In [9]:
for i in range(len(file_names)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

1-10-24
['A16885471', 'A17045592', 'A16893988', 'A16848913', 'A1770338', 'A16843394', 'A16796398', 'A16244966', 'A16219576', 'A17300170']
1-12-24
['A16843394', 'A16244966', 'A16706398', 'A16219576', 'A16688064', 'A17300170']
1-8-24
['A16217627', 'A16843394', 'A16272305', 'A17587027', 'A16244966', 'A16735067', 'A17045592', 'A16329013', 'A16893988', 'A17480805', 'A16848913', 'A16942072', 'A17655426', 'A16219576', 'A16796398', 'A17300170', 'A16865419']


## Week 2 Participation

Starting week 2, the deadline for in person and remote credit applies. 
- For in person credit, the participation form must be submitted with the correct passcode within 1 hour after the lecture (1PM for WI24)
- For remote credit, the participation form must be submitted with the passcode as "remote" within 48 hours of the class meeting

After loading in the spreadsheets, the for loop below will examine if the submission entry is within the deadline according to the passcode. If the passcode is correct and it is within the deadline, the record will be updated as "1" for in person credit and "1*" for remote credit. If the submission is missing, the record will be updated as "0". The for loop below will also spit out PIDs that doesn't match to any student on the roaster (usually due to typos from students). Usually I will just go back to the participation files and correct their PIDs for them. 

In [12]:
file_names = glob.glob('./Week2/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [13]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)
    
# Late add student: A17313198, adding credit for first 2 weeks lecture
roaster.loc[roaster['SIS User ID'] == 'A17313198'] = roaster.loc[roaster['SIS User ID'] == 'A17313198'].replace('0', '1')
#roaster[roaster['SIS User ID'] == 'A17313198']

1-17-24
['A16244966', 'A16843394', 'A18049536', 'A17300170']
1-19-24
['A17300170']


## Week 3 Attendance

In [8]:
file_names = glob.glob('./Week3/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [9]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if filename == '1-22-24':
                if (secretcode in entry['Secret Code'].tolist()[0]) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            
                elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
            else:
                if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            
                elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

1-22-24
[]
1-24-24
[]
1-26-24
[]


## Week 4 Attendance

In [10]:
file_names = glob.glob('./Week4/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [11]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if filename == '1-31-24':
                if (secretcode in entry['Secret Code'].tolist()[0]) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            
                elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
            else:
                if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
                elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                    roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

1-29-24
[]
1-31-24
[]
2-2-24
[]


## Week 5 Attendance 

In [12]:
file_names = glob.glob('./Week5/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [13]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

2-5-24
[]
2-7-24
[]
2-9-24
[]


## Week 6 Attendance 

In [14]:
file_names = glob.glob('./Week6/*.xlsx')
file_names_loop = file_names.copy()
files = []
for file in file_names_loop:
    if file in project_day:
        file_names.remove(file)
        continue
    else:
        temp_file = pd.read_excel(file)
        temp_file = temp_file[temp_file.columns[:7]]
        temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
        temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
        temp_file['PID'] = temp_file['PID'].str.upper()
        temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
        temp_file['PID'] = temp_file['PID'].str.replace('.', '')
        temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
        temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
        temp_file['Secret Code'] = temp_file['Secret Code'].astype('string').str.replace('-', '')
        temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
        temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
        files.append(temp_file)

In [15]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

2-14-24
[]
2-16-24
[]


## Week 7 Attendance

In [16]:
file_names = glob.glob('./Week7/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
    temp_file = temp_file.rename(columns = {
        'Todays *secret* code (sharing this code with someone not in class, or getting this code from someone else, is a violation of academic integrity)': 'Secret Code'
    })
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [17]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

2-21-24
[]
2-23-24
[]


## Week 8 Attendance

In [18]:
file_names = glob.glob('./Week8/*.xlsx')
file_names_loop = file_names.copy()
files = []
for file in file_names_loop:
    if file in project_day:
        file_names.remove(file)
        continue
    else:
        temp_file = pd.read_excel(file)
        temp_file = temp_file[temp_file.columns[:6]]
        temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
        temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
        temp_file['PID'] = temp_file['PID'].str.upper()
        temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
        temp_file['PID'] = temp_file['PID'].str.replace('.', '')
        temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
        temp_file = temp_file.rename(columns = {
        'Todays *secret* code (sharing this code with someone not in class, or getting this code from someone else, is a violation of academic integrity)': 'Secret Code'
        })
        temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
        temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
        temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
        files.append(temp_file)

In [19]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

2-28-24
[]
3-1-24
[]


## Week 9 Attendance

In [20]:
file_names = glob.glob('./Week9/*.xlsx')
file_names_loop = file_names.copy()
files = []
for file in file_names_loop:
    if file in project_day:
        file_names.remove(file)
        continue
    else:
        temp_file = pd.read_excel(file)
        temp_file = temp_file[temp_file.columns[:6]]
        temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
        temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
        temp_file['PID'] = temp_file['PID'].str.upper()
        temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
        temp_file['PID'] = temp_file['PID'].str.replace('.', '')
        temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
        temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
        temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
        temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
        files.append(temp_file)

In [21]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

3-6-24
[]
3-8-24
[]


## Week 10 Attendance

In [25]:
file_names = glob.glob('./Week10/*.xlsx')
files = []
for file in file_names:
    temp_file = pd.read_excel(file)
    temp_file = temp_file[temp_file.columns[:6]]
    temp_file['Timestamp'] = pd.to_datetime(temp_file['Timestamp'])
    temp_file['PID'] = temp_file['PID'].astype('string').str.replace('-', '')
    temp_file['PID'] = temp_file['PID'].str.upper()
    temp_file['PID'] = temp_file['PID'].apply(lambda x: 'A' + x if not x.startswith('A') and not x.startswith('a') else x)
    temp_file['PID'] = temp_file['PID'].str.replace('.', '')
    temp_file['PID'] = temp_file['PID'].str.replace(' ', '')
    temp_file['Secret Code'] = temp_file['Secret Code'].str.lower()
    temp_file['Secret Code'] = temp_file['Secret Code'].str.replace(' ', '')
    temp_file = temp_file.drop_duplicates(subset = ['PID'], keep = 'last')
    files.append(temp_file)

In [26]:
for i in range(len(files)):
    file = files[i]
    filename = file_names[i].split('/')[-1][:-5]
    missing_ID = []
    dl_sc = pd.to_datetime(filename + ' 13:00:00')
    dl_rm = dl_sc + pd.Timedelta(hours = 47)
    secretcode = secret_code[filename]

    roaster[filename] = ['0' for j in range(len(roaster))]
    for ID in file['PID']:
        if ID in roaster['SIS User ID'].dropna().tolist():
            entry = file[file['PID'] == ID]
            
            if (secretcode in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_sc).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
            elif (remote in entry['Secret Code'].tolist()) and (entry['Timestamp'] < dl_rm).tolist()[0]:
                roaster.loc[roaster['SIS User ID'] == ID, filename] = '1*'
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

3-11-24
[]
3-13-24
[]
3-15-24
[]


## Project Day Attendance

In [14]:
for file in project_day:
    missing_ID = []
    temp_file = pd.read_excel(file)
    filename = file.split('/')[-1][:-5]
    PIDs = temp_file[['PID #1', 'PID #2', 'PID #3', 'PID #4', 'PID #5', 'PID #6']]
    PIDs = PIDs.fillna('A99')
    for col in PIDs.columns:
        PIDs[col] = PIDs[col].astype('string').str.replace('-', '')
        PIDs[col] = PIDs[col].str.upper()
        PIDs[col] = PIDs[col].apply(lambda x: 'A' + x if pd.notna(x) and not x.startswith('A') and not x.startswith('a') else x)
        PIDs[col] = PIDs[col].str.replace('.', '')
        PIDs[col] = PIDs[col].str.replace(' ', '')

    PIDs = PIDs.values.flatten()
    PIDs = [str(i) for i in PIDs if str(i) != 'nan']
    
    for ID in PIDs:
        if ID in roaster['SIS User ID'].dropna().tolist():
            roaster.loc[roaster['SIS User ID'] == ID, filename] = '1'
        elif ID == 'A99':
            continue
        else:
            missing_ID.append(ID)
    
    print(filename)
    print(missing_ID)

2-12-24
["A(HASN'TCONTACTEDUSYET)"]
2-26-24
[]
3-4-24
['AWEDIDTHEPICTUREWITHJUSTTHREEMEMBERSBECAUSEALEXAWORKEDVIACALL:)', 'A1831529']


In [15]:
roaster['In Person Attendance'] = roaster.apply(lambda row: (row == '1').sum(), axis=1)
roaster['Remote Attendance'] = roaster.apply(lambda row: (row == '1*').sum(), axis=1)


In [27]:
#roaster.drop(columns = ['ID', 'SIS Login ID', 'Section']).to_csv('pilot participation 0320.csv', index = False)